# Chaining Functions

Running one prompt at a time can produce fantastic results! Sometimes you need several outputs from your AI copilot though. Of course you can simply call different skills in sequence in what is known as a prompt chain.

The simplest version of a prompt chain would be implemented by calling InvokeAsync for each function in the order you wanted them to execute.

A more interesting version of chaining allows you to use the output of one function to feed the next function thereby modifying the final result. Let's take a look at one of these more complex prompt chains.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

// load settings specific to you!
MySettings settings = Settings.LoadFromFile();

// Configure AI backend used by the kernel
var builder = new KernelBuilder();
if (settings.Type == "azure")
    builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.WithOpenAITextCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

The IKernel.RunAsync function allows you to provide one or more variables using a ContextVariables object and then any number of Semantic Kernel Functions in the order you want them to execute.

The functions can be any combination of any supported types of functions including semantic, inline, or native functions. Only native functions allow you to modify the ContextVariables object though. This is an important capability in many prompt chaining solutions.

Run the following example.

In [23]:
using Microsoft.SemanticKernel.Orchestration;
using System.IO;

#!import Plugins/ChaosPlugin.cs
#!import Plugins/MadLibPlugin.cs

// you can use any combination of semantic, native, or inline functions in a chain
var pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "Plugins");
var puzzlePlugin = kernel.ImportSemanticSkillFromDirectory(pluginsDirectory, "PuzzlePlugin");

var madLibPlugin = kernel.ImportSkill(new MadLibPlugin(kernel));
var chaosPlugin = kernel.ImportSkill(new ChaosPlugin(kernel));

// define your desired parameters needed by the chain of functions.
// some variables will be populated by the native functions in the chain as well!
ContextVariables pipelineContext = new();
pipelineContext["madLibTheme"] = "rock concert";

// pass your prompt to your OpenAI instance and retrieve the completion
var completion = await kernel.RunAsync(
    pipelineContext, 
    chaosPlugin["GenerateRandomNumbers"], 
    chaosPlugin["GenerateRandomWords"], 
    madLibPlugin["GenerateMadLib"], 
    puzzlePlugin["FillTheBlanksFunction"]);

Console.WriteLine(completion);

randomNumbers: 4,4,4
randomWords:  Adjectives: bright, tall, soft, warm; Nouns: tree, lake, chair, dog; Verbs: laugh, swim, fly, eat
madLib:  The ___(adjective)___ crowd was ___(verb)___ to the ___(adjective)___ rock concert. The ___(adjective)___ band was ___(verb)___ing their ___(adjective)___ songs on the ___(noun)___. Everyone was ___(verb)___ing and ___(verb)___ing to the ___(noun)___ music. The ___(noun)___ was filled with ___(noun)___ and excitement!

The bright crowd was laughing to the tall rock concert. The soft band was playing their warm songs on the chair. Everyone was eating and swimming to the lake music. The room was filled with dogs and excitement!


There's actually quite a bit of code in this solution and it's spread across a few different files, but we'll walk through it here as best we can.

First, we'll load all of the plugins we'll need.
- The Puzzle Plugin has just one semantic function (FillTheBlanksFunction) defined. It's job is to take a list of random adjectives, nouns, and verbs and use them to populate a mad lib template. We import the Puzzle Plugin into the Semantic Kernel by loading it it's location in our folder structure.
- The MadLib Plugin generates the mad lib template to be used by the Puzzle Plugin. The MadLib Plugin only has one function as well, but it's a native function. This function requires a theme for the generated mad lib as well as a list of numbers so it knows how many adjectives, nouns, and verb blanks to create in the mad lib. As a native plugin, we'll just create an instance of the MadLibPlugin class and import it into the Semantic Kernel. 
- Finally the Chaos Plugin - which is another native plugin, but it has two functions. The default behavior of the GenerateRandomNumbers function is to generate three integers between 0 and 5. We'll use these defaults because we only need three integers (adjectives, nouns, and verbs) and to keep our mad libs somewhat small.

We are using the ContextVariables object to pass values into the first function as well as between functions. The only variable we are required to initialize is the madLibTheme variable - which we defaulted to "rock concert". The other variables will actually be created within the native plugin functions. Open the ChaosPlugin.cs and the MadLibPlugin.cs to see where they are setting context variables. Open the semantic function FillTheBlanksFunction to see how it consumes the context variables it needs.
- *NOTE: Feel free to change the madLibTheme context variable to generate a mad lib about something you think is interesting. :) You could also initialize context variables for lowerBound and upperBound parameters used by the GenerateRandomNumbers function. You can set them to higher integer values in order to force the AI to generate larger mad libs with more blanks to fill out.*

The second to last line of code is where it all comes together. The Semantic Kernel's RunAsync method takes the Context Variables object (which just has the one madLibTheme context variable initially) and then runs each of the specified functions in order. The context variables are made available to all of the functions.

# Exercise

Create a new prompt chain solution targeting international students learning English. You want to surprise them every day with a list of words and their definitions.

*Hint: Use the Chaos Plugin's Generate Random Numbers and Generate Random Words functions. You'll have to write your own semantic, inline, or native function to fetch the definition of words.*